In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Assign `animals` to refer to the `db` object for read operations
animals = db

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Coral Stewart Project Two Dashboard'))),
    html.Hr(),
    html.Img(id='customer-image', src='data:image/png;base64,{}'.format(encoded_image.decode()), alt='customer image'),
    html.Div(
        # Radio buttons for filter options
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,  # Display 10 rows per page
        sort_action='native',  # Enable sorting
        row_selectable='single',  # Allow single row selection
        style_table={'overflowX': 'auto'},  # Add horizontal scrolling if needed
    ),
    html.Br(),
    html.Hr(),
    html.Div(id='map-id', className='col s12 m6'),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to filter interactive data table with MongoDB queries

@app.callback(
    [Output('datatable-id', 'data'), Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    try:
        # Water Rescue filter
        if filter_type == 'water':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
            }))
        # Mountain or Wilderness Rescue filter
        elif filter_type == 'mount':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
            }))
        # Disaster Rescue or Individual Tracking filter
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher", "German Shepard", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
            }))
        # Reset to no filter
        else:
            df = pd.DataFrame.from_records(animals.read({}))
        
        # Handle case where DataFrame is empty
        if df.empty:
            return [], []
        
        # Drop '_id' column if it exists
        if '_id' in df.columns:
            df.drop(columns=['_id'], inplace=True)

        # Return updated columns and data
        columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data = df.to_dict('records')

        return data, columns

    except Exception as e:
        print(f"Error updating DataTable: {e}")
        return [], []

# Callback to update the graph based on the table data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    return [dcc.Graph(figure=px.histogram(dff, x='breed'))]

# Callback to highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # Check if selected_columns is None or empty
    if selected_columns is None or len(selected_columns) == 0:
        return []  # Return an empty list if no columns are selected

    # Apply style to selected columns
    return [{
        'if': {'column_id': i},
        'backgroundColor': '#D2F3FF'  # 'background_color' should be 'backgroundColor'
    } for i in selected_columns]

# Callback to update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if viewData is None or index is None or len(index) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index is not None else 0
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row, 9])])
            ])
        ])
    ]

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



Hello, Animal Shelter
in read method...
Dash app running on http://127.0.0.1:16630/
in read method...
in read method...
